In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
import os

In [2]:
model_path = "model_jit.pth"
model = torch.jit.load(model_path)
model.eval()

RecursiveScriptModule(
  original_name=FireDetectionModel
  (model): RecursiveScriptModule(
    original_name=ResNet
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
    (relu): RecursiveScriptModule(original_name=ReLU)
    (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
    (layer1): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=Bottleneck
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (conv2): RecursiveScriptModule(original_name=Conv2d)
        (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
        (conv3): RecursiveScriptModule(original_name=Conv2d)
        (bn3): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu): RecursiveScriptModule(original_name=ReLU)
        (downsample): RecursiveScriptModule(
          original_name=Sequential
  

In [3]:
# Définir les transformations pour les images selon l'entraînement du modèle
test_transforms = transforms.Compose([
        transforms.RandomResizedCrop((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

# Charger les données
test_dataset_path = "test"
test_dataset = datasets.ImageFolder(test_dataset_path, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [4]:
def evaluate_model(model, dataloader, criterion):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            # Envoyer les données sur GPU si disponible
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()
                model = model.cuda()

            # Prédictions
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Calcul de l'accuracy
            _, preds = torch.max(outputs, 1)
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)

    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy


In [18]:
# GOOGLE NEt
def evaluate_model(model, dataloader, criterion):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            # Envoyer les données sur GPU si disponible
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()
                model = model.cuda()

            # Prédictions
            outputs = model(inputs)

            # Gestion spécifique pour GoogleNet
            if hasattr(outputs, "logits"):
                logits = outputs.logits
            else:
                logits = outputs  # Pour les autres modèles

            # Calcul de la perte
            loss = criterion(logits, labels)
            total_loss += loss.item()

            # Calcul de l'accuracy
            _, preds = torch.max(logits, 1)
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)

    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy



In [5]:
# Définir la fonction de perte (CrossEntropy pour la classification multi-classes)
criterion = F.cross_entropy

# Calculer la perte et l'accuracy
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Test Loss: 0.2039
Test Accuracy: 0.9433
